In [1]:
#download libraries and packages
# !pip install imutils
# !pip install opencv-python
# !pip install tensorflow
# !pip install tensorflow_hub
# !pip install matplotlib

from imutils import paths
# from PIL import Image
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import random
import os
import tensorflow_hub as hub

2023-08-15 13:22:41.656361: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-15 13:22:41.703591: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-15 13:22:42.934345: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
batch_size = 16
img_height = 200 
img_width = 200 
training_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'train', # directory path
    validation_split = None, 
    subset = None, 
    image_size = (img_height, img_width), # image size of height and width
    batch_size = batch_size # batch size
)

Found 57418 files belonging to 7 classes.


2023-08-15 13:22:53.208199: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-15 13:22:53.227908: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-15 13:22:53.227998: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
testing_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'test', # directory path
    validation_split = None, 
    subset = None, 
    image_size = (img_height, img_width), # image size of height and width
    batch_size = batch_size # batch size
)

Found 14356 files belonging to 7 classes.


In [4]:
# One-hot encode the labels
num_classes = len(training_ds.class_names)
def one_hot_encode(image, label):
    label = tf.one_hot(label, num_classes)
    return image, label

# # Apply one-hot encoding to the dataset
training_ds = training_ds.map(one_hot_encode)
testing_ds = testing_ds.map(one_hot_encode)

In [5]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D



img_channels = 3
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, img_channels))
for layer in base_model.layers:
    layer.trainable = False

effnet = Sequential()
effnet.add(base_model)
effnet.add(GlobalAveragePooling2D())
effnet.add(Dense(256, activation='relu'))
effnet.add(Dense(7, activation='softmax'))
effnet.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_saved = effnet.fit(training_ds, validation_data=testing_ds, epochs=20)
# effnet.save("/Users/janrauhl/Documents/Dissertation/FER/efficientnet_base.h5")

Epoch 1/20


InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]]
	 [[categorical_crossentropy/softmax_cross_entropy_with_logits/Shape_2/_6]]
  (1) INVALID_ARGUMENT:  Unknown image file format. One of JPEG, PNG, GIF, BMP required.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_16670]